In [3]:
%matplotlib inline

In [4]:
import matplotlib.pyplot as plt
import numpy as np 
import time
import pyvisa as visa

In [5]:
VISA_RM = visa.ResourceManager('@py')

In [157]:
class RigolSupply():
    def __init__(self, address, n_ch, visa_resource_manager=VISA_RM):
        resource_str = f'TCPIP0::{address:s}::INSTR'
        print(resource_str)
        self.resource = VISA_RM.open_resource(resource_str, write_termination='\n', read_termination='\n')

        self.write = self.resource.write
        self.query = self.resource.query
        
        self.n_ch = n_ch
        
        self.voltages = np.zeros(n_ch)
        self.currents = np.zeros(n_ch)
        
        self.ovp_lim = np.zeros(n_ch)
        self.ocp_lim = np.zeros(n_ch)
        
    @property
    def IDN(self):
        return self.query("*IDN?")
    
    @property
    def IP(self):
        return self.query(":SYSTem:COMMunicate:LAN:IPADdress?")
    
    @property
    def IDENTITY(self):
        return f"IDN: {self.IDN.split(',')[-2]} IP: {self.IP}"
        
    def ask(self, question, verbose=False):
        response = self.query(question)
        if verbose:
            print("Question: {0:s} - Response: {1:s}".format(question, str(response)))
        return response
    
    def tell(self, statement):
        return self.write(statement)
    
    def enable_output(self, ch):
        return self.tell(f"OUTP:STAT CH{ch},ON")
    
    def disable_output(self, ch):
        return self.tell(f"OUTP:STAT CH{ch},OFF")

    def set_ocp(self, ocp):
        for ch in range(n_ch):
            self.tell(f":OUTP:OCP:VAL CH{ch},{ocp[ch]}")
    
    def get_ocp(self):
        return [self.query(f":OUTP:OCP:VAL? CH{ch}") for ch in range(self.n_ch)]


def apply_to_all(supply_hanlers, ask=None, tell=None):
    for supply in supply_handlers:
        if ask is not None:
            supply.ask(ask)
        if tell is not None:
            supply.tell(tell)
        

            
            
            
            
def power_on_fpga(supply_handlers, fpga_pair):
    supply_handlers[fpga_pair[0]].enable_output(fpga_pair[1])
    
def power_off_fpga(supply_handlers, fpga_pair):
    supply_handlers[fpga_pair[0]].disable_output(fpga_pair[1])
    
def power_cycle_fpga(supply_handlers, fpga_pair, sleepytime=1):
    power_off_fpga(supply_handlers, fpga_pair)
    time.sleep(sleepttime)
    power_on_fpga(supply_handlers, fpga_pair)
    
def power_cycle_all_supplies(supply_handlers):
    for supply in supply_handlers:
        for ch in range(supply.n_ch):
            print(supply.enable_output(ch+1))
            time.sleep(1)
            print(supply.disable_output(ch+1))

def report_status(supply_handlers):
    for supply in supply_handlers:
        print(supply.IDENTITY)
        print(f"\tChannel\t| Status\t| V\t\t| I (A)\t\t| P (W)  ")
        channel_stats = []
        channel_out_vals = []
        for ch in range(supply.n_ch):
            stat = supply.query(f"OUTP:STAT? CH{ch+1}")
            vals = supply.query(f"MEASure:ALL? CH{ch+1}")
            vals = vals.split(',')
            vals = [float(i) for i in vals]
            print(f"\t{ch+1:7d}\t| {stat:8s}\t| {vals[0]:3.4f}\t| {vals[1]:3.4f}\t| {vals[2]:3.4f}")
        print()
        
def power_up_all(supply_handlers, verbose=False):
    for supply in supply_handlers:
        for ch in range(supply.n_ch):
            supply.enable_output(ch+1)        
    if verbose:
        report_status(supply_handlers)
    
    
def power_down_all(supply_handlers, verbose=False):
    for supply in supply_handlers:
        for ch in range(supply.n_ch):
            supply.enable_output(ch+1)
    if verbose:
        report_status(supply_handlers)

In [158]:
supply_IPs = ["10.10.1.50", 
              "10.10.1.51", 
              "10.10.1.52", 
              "10.10.1.53"]

supply_channels = [2, 2, 3, 2]
fpga_pair = [1, 2]

AFE_power_lines = [[0, 1], [0,2], 
                   [1, 0], #Second channel iss the FPGA.
                   [2, 0], [2, 2],
                   [3, 0], [3, 1]
                  ]

supply_handlers = [RigolSupply(ip, n_ch) for ip,n_ch in zip(supply_IPs, supply_channels)]

TCPIP0::10.10.1.50::INSTR
TCPIP0::10.10.1.51::INSTR
TCPIP0::10.10.1.52::INSTR
TCPIP0::10.10.1.53::INSTR


In [159]:
report_status(supply_handlers)

IDN: DP8E203100159 IP: 10.10.1.50
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 5.4110	| 0.0025	| 0.0140
	      2	| ON      	| 1.9010	| 0.0000	| 0.0000

IDN: DP8E203700164 IP: 10.10.1.51
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 5.4000	| 0.0012	| 0.0060
	      2	| OFF     	| 0.0000	| 0.0000	| 0.0000

IDN: DP8B212500667 IP: 10.10.1.52
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 4.9993	| 0.0000	| 0.0000
	      2	| ON      	| 3.4531	| 0.0000	| 0.0000
	      3	| ON      	| 2.4015	| 0.0000	| 0.0000

IDN: DP8E214200286 IP: 10.10.1.53
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 0.4980	| 0.0002	| 0.0000
	      2	| ON      	| 3.6000	| 0.0000	| 0.0000



In [170]:
power_up_all(supply_handlers, verbose=True)

IDN: DP8E203100159 IP: 10.10.1.50
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 5.4120	| 0.0289	| 0.1560
	      2	| ON      	| 1.9000	| 0.2870	| 0.5450

IDN: DP8E203700164 IP: 10.10.1.51
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 5.4000	| 0.0402	| 0.2170
	      2	| ON      	| 3.4480	| 1.3310	| 4.5890

IDN: DP8B212500667 IP: 10.10.1.52
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 4.9991	| 0.0000	| 0.0000
	      2	| ON      	| 3.4531	| 0.0000	| 0.0000
	      3	| ON      	| 2.4015	| 0.3768	| 0.9050

IDN: DP8E214200286 IP: 10.10.1.53
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 0.4980	| 0.0002	| 0.0000
	      2	| ON      	| 3.6000	| 0.1610	| 0.5800



In [161]:
supply_handlers[-1].ask("MEAS:ALL? CH1")
supply_handlers[-1].tell("APPLY CH1,0.5,0.01")
supply_handlers[-1].ask("MEAS:ALL? CH1")

'0.498,0.0002,0.000'

In [169]:
for supply in supply_handlers:
    print(supply.IDENTITY)
    for ch in range(supply.n_ch):
        ocp = supply.ask(f"OUTP:OCP:val? CH{ch+1}")
        print(f"CH{ch+1} {ocp}")
    print()

IDN: DP8E203100159 IP: 10.10.1.50
CH1 0.3000
CH2 1.500

IDN: DP8E203700164 IP: 10.10.1.51
CH1 0.3000
CH2 4.000

IDN: DP8B212500667 IP: 10.10.1.52
CH1 0.285
CH2 3.300
CH3 3.000

IDN: DP8E214200286 IP: 10.10.1.53
CH1 1.1000
CH2 2.000



In [163]:
report_status(supply_handlers)

IDN: DP8E203100159 IP: 10.10.1.50
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 5.4110	| 0.0025	| 0.0140
	      2	| ON      	| 1.9010	| 0.0000	| 0.0000

IDN: DP8E203700164 IP: 10.10.1.51
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 5.4000	| 0.0012	| 0.0060
	      2	| ON      	| 3.4480	| 1.3690	| 4.7200

IDN: DP8B212500667 IP: 10.10.1.52
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 4.9994	| 0.0000	| 0.0000
	      2	| ON      	| 3.4531	| 0.0000	| 0.0000
	      3	| ON      	| 2.4015	| 0.0000	| 0.0000

IDN: DP8E214200286 IP: 10.10.1.53
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 0.4980	| 0.0002	| 0.0000
	      2	| ON      	| 3.6000	| 0.0000	| 0.0000



In [10]:
power_on_all_supplies(supply_handlers)

RIGOL TECHNOLOGIES,DP821A,DP8E203100159,00.01.14
2
(17, <StatusCode.success: 0>)
(17, <StatusCode.success: 0>)
RIGOL TECHNOLOGIES,DP821A,DP8E203700164,00.01.14
2
(17, <StatusCode.success: 0>)
(17, <StatusCode.success: 0>)
RIGOL TECHNOLOGIES,DP832A,DP8B212500667,00.01.16
3
(17, <StatusCode.success: 0>)
(17, <StatusCode.success: 0>)
(17, <StatusCode.success: 0>)
RIGOL TECHNOLOGIES,DP821A,DP8E214200286,00.01.16
2
(17, <StatusCode.success: 0>)
(17, <StatusCode.success: 0>)
